In [19]:
from azure.ai.ml import MLClient
from azure.identity import (
    DefaultAzureCredential,
)
from azure.ai.ml.entities import (
    AmlCompute, Model, Environment, BuildContext, CodeConfiguration, 
    ManagedOnlineEndpoint, ManagedOnlineDeployment, OnlineRequestSettings, ProbeSettings
)
from azure.ai.ml.constants import AssetTypes

import time, sys, json, os
from IPython.display import display, JSON

In [20]:
credential = DefaultAzureCredential()

workspace_ml_client = MLClient.from_config(
    credential
)

Found the config file in: /config.json


### Register the model asset in workspace

In [21]:
model_name = "tinyllama-gguf"

# Make the models query fail-safe
models = workspace_ml_client.models.list()
model = next((m for m in models if m.name == model_name), None)

if model is not None:
    print(f"Model {model.name} already exists.")
else:
    # Register the foundation model from local as Model asset in ml workspace
    model = Model(
        path="models",
        type=AssetTypes.CUSTOM_MODEL,
        name=model_name,
        description="tinyllamagguf - gguf format model",
    )

    workspace_ml_client.create_or_update(model)

Model tinyllama-gguf already exists.


### Register the environment asset in workspace

In [22]:
env_name = "tinyllama-gguf-env"

# Make the environments query fail-safe
envs = workspace_ml_client.environments.list()
env = next((e for e in envs if e.name == env_name), None)

# Flag to indicate if the environment is modified. 
# Set it manually:: If True, old env version is used. If False, new env version is created.
is_env_earlierone = True

if env is not None and is_env_earlierone:
    print(f"Environment {env_name} already exists.")
else:
    print("new")
    # Create a new environment based on the curated one
    custom_env = Environment(
        name=env_name,
        description="Custom environment with additional dependencies",
        build=BuildContext(path="../env/cpu")
    )

    # Register the custom environment
    workspace_ml_client.environments.create_or_update(custom_env)

Environment tinyllama-gguf-env already exists.


### Create managed online endpoint

In [23]:
# Create online endpoint - endpoint names need to be unique in a region, hence using timestamp to create unique endpoint name
online_endpoint_name = "tinyllama-gguf-ep"

In [24]:
# managed endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Online endpoint for tinyllama gguf",
    auth_mode="key",
)

In [25]:
# managed endpoint create async call
workspace_ml_client.begin_create_or_update(endpoint).wait()

### Create managed online deployment for the endpoint

In [26]:
deployment_name = "deploy01"
model = f"{model_name}@latest"
env = f"{env_name}@latest"

In [27]:
# managed endpoint deployment
demo_deployment = ManagedOnlineDeployment(
    name=deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    environment=env,
    code_configuration=CodeConfiguration(
        code="../onlinescoring",
        scoring_script="score.py",
    ),
    instance_type="Standard_F16s_v2",
    instance_count=1,
    request_settings=OnlineRequestSettings(
        request_timeout_ms=120000,
    ),
    liveness_probe=ProbeSettings(
        initial_delay=600
    ),
)

In [28]:
# managed endpoint deplyment create async call
workspace_ml_client.online_deployments.begin_create_or_update(deployment=demo_deployment).wait()

Check: endpoint tinyllama-gguf-ep exists


...................................................................

In [ ]:
# update traffic to the deployment for 100%
endpoint.traffic = {deployment_name: 100}
workspace_ml_client.begin_create_or_update(endpoint).result()

### Test endpoint with sample data

In [30]:
# score the request file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="../payload/request1.json",
)

print(response)

{"output": "1. Champs-Elys\u00e9es\n2. Eiffel Tower\n3. Notre Dame Cathedral\n4. Arc de Triomphe\n5. Louvre Museum\n6. Giverny Garden\n7. The Seine River\n8. Montmartre\n9. Jardin des Plantes\n10. Mus\u00e9e d'Orsay\n\nRemember to always check the weather and safety rules before visiting these places, especially if you plan on seeing any landmarks or monuments."}


In [31]:
# score the request file using the online endpoint with the azureml endpoint invoke method
response = workspace_ml_client.online_endpoints.invoke(
    endpoint_name=online_endpoint_name,
    request_file="../payload/request3.json",
)

print(response)

{"output": "The capital of India is New Delhi."}
